In [ ]:
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from telemetry.telemetry_data_preprocessing import request_data, telemetry_computations, telemetry_preprocessing
import fastf1

computations = telemetry_computations()
request = request_data()

driver_number = "1"
country_name = "Bahrain"
year = "2023"

sessions_data = []

for session in sessions_data:
    match session["session_name"]:
        case "Qualifying":
            
            params = {
                "driver_number" : driver_number,
                "session_key" : session["session_key"]
            }
            qualifying_car_data = request.get_car_data(params=params)
            print("got quali car data..")
            qualifying_location_data = request.get_location_data(params=params)
            print("got quali location data..")
            qualifying_laps_data = request.get_laps_data(params=params)
            print("got quali laps data..")
            
        case "Race":
            
            params = {
                "driver_number" : driver_number,
                "session_key" : session["session_key"]
            }
            race_car_data = request.get_car_data(params=params)
            print("got race car data..")
            race_location_data = request.get_location_data(params=params)
            print("got race location data..")
            race_laps_data = request.get_laps_data(params=params)
            print("got race laps data..")
            
            
df_qualifying_car_data = pd.DataFrame(qualifying_car_data)
## ....
## ....
## ....
           
df_qualifying_car_data = df_qualifying_car_data.iloc[:, 1:-2]

df_qualifying_car_data['date'] = pd.to_datetime(df_qualifying_car_data['date'], format='ISO8601')

df_qualifying_car_data['drs'] = df_qualifying_car_data['drs'].apply(lambda x: 1 if x in [10, 12, 14] else 0)


df_qualifying_laps_data["date_start"] = pd.to_datetime(df_qualifying_laps_data["date_start"], format='ISO8601')

df_qualifying_laps_data["lap_duration"] = pd.to_timedelta(df_qualifying_laps_data["lap_duration"], unit="s")


df_race_car_data = df_race_car_data.iloc[:, 1:-2]

df_race_car_data['date'] = pd.to_datetime(df_race_car_data['date'], format='ISO8601')

df_race_car_data['drs'] = df_race_car_data['drs'].apply(lambda x: 1 if x in [10, 12, 14] else 0)

df_race_laps_data["date_start"] = pd.to_datetime(df_race_laps_data["date_start"], format='ISO8601')

df_race_laps_data["lap_duration"] = pd.to_timedelta(df_race_laps_data["lap_duration"], unit="s")

df_race_location_data["date"] = pd.to_datetime(df_race_location_data['date'], format='ISO8601')

In [ ]:
df_telemetry = telemetry_preprocessing().join_frames_by_date(df_race_location_data, df_race_car_data)

df_telemetry = df_telemetry[~df_telemetry["date"].isna()]

df_telemetry_test  = df_telemetry.copy()

df_telemetry_test["lap"] = 0


i = 1
for index, row in df_race_laps_data.iloc[1:10, :].iterrows(): 
    
        if i == 1:
            timestamp_end = row["date_start"]
            end_index = telemetry_preprocessing().data_closest_timestamp(timestamp_end, df_telemetry) 
            df_telemetry_test.loc[:end_index+1, "lap"] = i 
            timestamp_start = timestamp_end
            i += 1
            continue
        
        start_index = telemetry_preprocessing().data_closest_timestamp(timestamp_start, df_telemetry) 
        timestamp_end = row["date_start"]
        end_index = telemetry_preprocessing().data_closest_timestamp(timestamp_end, df_telemetry) 
        df_telemetry_test.loc[start_index:end_index+1, "lap"] = i 
        
        timestamp_start = timestamp_end
        
        i += 1
        
        
ref_time = df_telemetry_test["date"].iloc[0]  
df_telemetry_test["delta"] = df_telemetry_test["date"].apply(
    lambda x: (x - ref_time).total_seconds()
)


lon, lat = telemetry_computations().compute_accelerations(telemetry=telemetry)

In [ ]:
telemetry.DRS = telemetry.DRS.apply(lambda x: 1 if x in [10, 12, 14] else 0)

plt.plot(telemetry.DRS)